In [1]:
import pandas as pd
from google.colab import drive

# Mount Google Drive (if not already mounted)
drive.mount('/content/drive')

# Define file paths/URLs
github_emissions_url = "https://raw.githubusercontent.com/apownukepcc/datathon-spring-2025/main/emissions_data_updated.csv"
github_weather_url   = "https://raw.githubusercontent.com/apownukepcc/datathon-spring-2025/main/weather.csv"
output_file = "/content/drive/My Drive/final_dataset_with_weather.csv"

# Load the emissions dataset
try:
    emissions_data = pd.read_csv(github_emissions_url)
    print("Emissions dataset successfully loaded from GitHub.")
except Exception as e:
    print(f"Error loading emissions dataset: {e}")
    exit()

# Print columns to verify structure.
print("Emissions Data Columns:", emissions_data.columns.tolist())

# In this dataset, use 'TimeStamp' as the date column.
if "TimeStamp" not in emissions_data.columns:
    print("Error: 'TimeStamp' column not found in emissions data.")
    exit()
else:
    # Rename TimeStamp to date for consistency.
    emissions_data.rename(columns={"TimeStamp": "date"}, inplace=True)

# Ensure that required columns exist in emissions data.
required_columns = ["date", "Source", "Parameter", "Value"]
for col in required_columns:
    if col not in emissions_data.columns:
        print(f"Error: Required column '{col}' not found in emissions data.")
        exit()

# Pivot the emissions dataset so that each unique 'Parameter' becomes its own column.
try:
    final_data = emissions_data.pivot_table(index=["date", "Source"],
                                            columns="Parameter",
                                            values="Value").reset_index()
    print("Emissions data successfully pivoted.")
except Exception as e:
    print(f"Error during pivoting the data: {e}")
    exit()

# Define the desired parameters to include as separate columns.
desired_params = [
    'UNITONBT', 'SO2TONS', 'HEATINBA', 'NH3TONS',
    'UNITONBA', 'HEAT_QA', 'HEATINBT', 'GFLOW_BA',
    'NOXTONS', 'LOADMWBA', 'COTONS', 'LOADMWBT'
]

# Ensure each desired parameter is in the pivoted data; if missing, add as empty.
for param in desired_params:
    if param not in final_data.columns:
        final_data[param] = pd.NA

# Reorder columns so that 'date' and 'Source' come first.
final_data = final_data[['date', 'Source'] + desired_params]

# Calculate Emissions_Load for selected emission parameters relative to the load parameter (LOADMWBA).
# Here we compute: Emissions_Load = (parameter value) / LOADMWBA for each emission parameter.
emission_params = ['SO2TONS', 'NOXTONS', 'COTONS']
if 'LOADMWBA' in final_data.columns:
    for param in emission_params:
        if param in final_data.columns:
            # Using .loc to avoid SettingWithCopyWarning
            final_data.loc[:, f'Emissions_Load_{param}'] = final_data[param] / final_data['LOADMWBA']
else:
    print("Warning: 'LOADMWBA' column not found. Skipping emissions load calculations.")

# Load the weather dataset.
try:
    weather_data = pd.read_csv(github_weather_url)
    print("Weather dataset successfully loaded from GitHub.")
except Exception as e:
    print(f"Error loading weather dataset: {e}")
    exit()

# Print weather dataset columns for verification.
print("Weather Data Columns:", weather_data.columns.tolist())

# Ensure weather data has the 'date' column. If needed, adjust accordingly.
if "date" not in weather_data.columns and "Date" in weather_data.columns:
    weather_data.rename(columns={"Date": "date"}, inplace=True)
if "date" not in weather_data.columns:
    print("Error: 'date' column not found in weather data.")
    exit()

# Merge the weather data into the final emissions dataset.
# Since the weather data does not have a 'Source' column, we merge on 'date' only.
try:
    final_data_with_weather = pd.merge(final_data, weather_data, on="date", how="left")
    print("Weather data successfully merged.")
except Exception as e:
    print(f"Error merging weather data: {e}")
    exit()

# Save the final merged dataset to a CSV file in Google Drive.
try:
    final_data_with_weather.to_csv(output_file, index=False)
    print(f"Final dataset with weather data saved to: {output_file}")
except Exception as e:
    print(f"Error saving final dataset: {e}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Emissions dataset successfully loaded from GitHub.
Emissions Data Columns: ['Plant', 'Source', 'Parameter', 'Units', 'TimeStamp', 'Value', 'Description']
Emissions data successfully pivoted.
Weather dataset successfully loaded from GitHub.
Weather Data Columns: ['date', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'pres']
Weather data successfully merged.
Final dataset with weather data saved to: /content/drive/My Drive/final_dataset_with_weather.csv
